In [1]:
import random
import string

In [31]:
class steckerbrett:
    def __init__(self, wiring_pairs):
        '''
        Initialising the Steckerbrett with the letter pairs
        
        wiring_pairs: list of tuples, for example, [('A', 'B'), ('C', 'D')]
        '''
        
        self.wiring = {}
        for a, b in wiring_pairs:
            self.wiring[a] = b
            self.wiring[b] = a # creating a dictionary with the pairs

    def swap(self, letter):
        # swapping only those specific pairs
        return self.wiring.get(letter, letter) # letter to swap

In [30]:
class rotor:
    def __init__(self, wiring, notch, position=0):
        # initialising a rotor 
        # based on Caesar ciphers
        self.wiring = wiring
        self.notch = notch # position where the rotor causes the next rotor to advance
        self.position = position # initial pos of rotor (0 by default)

    def forward(self, letter):
        # passes the letter on to the next rotor
        index = ((ord(letter) - ord('A')) + self.position) % 26
        encoded_letter = self.wiring[index]
        return chr((ord(encoded_letter) - ord('A') - self.position) % 26 + ord('A')) # reverse adjustment

    def back(self, letter):
        '''
        Passes the letter backward through the rotor - needed as the signal goes through the rotor
        in both directions during encryption and decryption
        '''
        
        index = (ord(letter) - ord('A') + self.position) % 26
        decoded_index = self.wiring.index(chr(index + ord('A')))
        return chr((decoded_index - self.position) % 26 + ord('A')) # reverse adjustment

    def rotation(self):
        self.position = (self.position + 1) % 26 # rotor rotates by 1 position each time
        return self.position == self.notch # True => the next rotor should rotate

In [32]:
class reflector:
    def __init__(self, wiring):
        self.wiring = wiring # reflector wiring

    def reflect(self, letter):
        index = ord(letter) - ord('A')
        return self.wiring[index]

In [45]:
class Enigma:
    def __init__(self, Steckerbrett, Rotors, Reflector):
        self.Steckerbrett = Steckerbrett
        self.Rotors = Rotors
        self.Reflector = Reflector
        self.initial_pos = [r.position for r in Rotors]

    def encrypt(self, letter):
        # first passes through the plugboard
        letter = self.Steckerbrett.swap(letter)
        
        # and then through the rotors
        for i in self.Rotors:
            letter = i.forward(letter)

        # reflection
        letter = self.Reflector.reflect(letter)

        # back through the rotors
        for i in reversed(self.Rotors):
            letter = i.back(letter)

        # back through the steckerbrett again
        letter = self.Steckerbrett.swap(letter)

        # rotation of rotors
        for r in self.Rotors:
            if not r.rotation():
                break

        return letter # encrypted!!

    def decrypt(self, letter):
        # resetting rotor positions before decrypting
        for i, r in enumerate(self.Rotors):
            r.position = self.initial_pos[i]
        # encrypting the letter as usual (works for decryption due to symmetry)
        return self.encrypt(letter)

In [38]:
# IGNORE
# created just to generate rotor wiring settings
a = list(string.ascii_uppercase)
random.shuffle(a)
print(''.join(a))

VYTGQIPRMHSWFCDJEUKALOXZNB


In [46]:
# Configurations

steckerbrett_settings = [('Q', 'Z'), ('P', 'M')]
rotor1 = rotor('TMFIAPLNRUBEGWJDKQXHCZVSOY', notch=16)  
rotor2 = rotor('AMFUSYCDWRZKLQGTJXVBOEIHPN', notch=4)
rotor3 = rotor('VYTGQIPRMHSWFCDJEUKALOXZNB', notch=21)
ref = reflector('FAUTKSIXMBNLZJWQOYCPVRHDEG')

Steckerbrett = steckerbrett(steckerbrett_settings)
enigma = Enigma(Steckerbrett, [rotor1, rotor2, rotor3], ref)

# Encryption

In [49]:
message = 'Das wetter in Berlin ist wahrscheinlich wunderbar heil Sahaana'
enc = ''.join(enigma.encrypt(j) if j.isalpha() else j for j in message)
print("Encrypted:", enc)

Encrypted: MNP IAGUGT VZ YTQXWC ECU MHLEVIELUDMNWN MAUYJSRHA CYRD IUNJAUT


# Decryption (Work in progress as the Enigma is sensitive to initial rotor positions)

In [50]:
dec = ''.join(enigma.decrypt(j) if j.isalpha() else j for j in enc)
print("Decrypted:", dec)

Decrypted: MIY THVNVU OF WUZBXQ SQN MLCSOTSCNJMIXI MHNWPADLH QWDJ TNIPHNU
